## **HRNet Backbone**

**Using**: [MMPretrain to import model](https://mmpretrain.readthedocs.io/en/latest/papers/hrnet.html) <br/>
**PyTorch Installation**: [Documentation](https://pytorch.org/)

**Some important commands for MMPretrain related APIs:**
1) `sudo apt-get update`
2) Now if `E: Unable to lock directory` appears:
    1) Check for running "apt-get" process using: `ps aux | grep apt-get`
    2) If yes, kill it using: `sudo kill -9 204456`
    3) Now try `sudo apt-get update` again
3) `sudo apt-get install libgl1-mesa-glx` (OpenCV library requires libGL.so.1, which is part of the OpenGL library)
4) Now activate your **venv** to start installing packages
5) `pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu` (for CPU based LINUX OS)
6) `pip install -U openmim && mim install "mmpretrain>=1.0.0rc8"`(Downloading MMpretrain library)
7)  Install other requirments like PyTorch, Open-cv, etc using standard pip commands  

In [8]:
# Pip command for pytorch for Linux - CPU

!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu


In [1]:
!pip install -U openmim && mim install "mmpretrain>=1.0.0rc8"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.4.0/index.html
  Using cached mmcv-2.2.0-cp310-cp310-linux_x86_64.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mmyolo 0.6.0 requires mmdet>=3.0.0, which is not installed.
mmyolo 0.6.0 requires mmcv<2.1.0,>=2.0.0rc4, but you have mmcv 2.2.0 which is incompatible.


In [3]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install torchsummary

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from mmpretrain import get_model
from torch.nn import functional as F
from torchsummary import summary

In [5]:
# Load the pretrained model
model = get_model('hrnet-w30_3rdparty_8xb32_in1k', pretrained=True).eval()


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmclassification/v0/hrnet/hrnet-w30_3rdparty_8xb32_in1k_20220120-8aa3832f.pth


Downloading: "https://download.openmmlab.com/mmclassification/v0/hrnet/hrnet-w30_3rdparty_8xb32_in1k_20220120-8aa3832f.pth" to /home/ram/.cache/torch/hub/checkpoints/hrnet-w30_3rdparty_8xb32_in1k_20220120-8aa3832f.pth


In [17]:
import torch
import torch.nn as nn

def print_model_summary(model, input_size):
    def register_hooks(module):
        def hook(module, input, output):
            num_params = sum(p.numel() for p in module.parameters())
            layer_name = f"{module.__class__.__name__}"
            # For named layers, you might want to include the layer name if available
            if hasattr(module, 'name') and module.name:
                layer_name = module.name
            # Handle outputs that might be lists or tuples
            if isinstance(output, (list, tuple)):
                output_shapes = [str(o.shape) for o in output]
                output_str = str(output_shapes)
            else:
                output_str = str(output.shape)
            print(f"{layer_name.ljust(30)} | "
                  f"Output Shape: {output_str.ljust(50)} | "
                  f"Parameters: {num_params}")
        
        hooks = []
        for name, module in model.named_modules():
            hooks.append(module.register_forward_hook(hook))
        return hooks
    
    # Print model summary
    print(f"{'Layer':<40}{'Output Shape':<50}{'Param #':<20}")
    print("="*110)
    
    # Dummy input to pass through the model
    dummy_input = torch.randn(1, *input_size)
    hooks = register_hooks(model)
    with torch.no_grad():
        model(dummy_input)
    
    # Remove hooks
    for hook in hooks:
        hook.remove()

# Define model and input size
model = get_model('hrnet-w30_3rdparty_8xb32_in1k', pretrained=True).eval()
input_size = (3, 640, 640)  # (channels, height, width)

# Print the model summary
print_model_summary(model, input_size)


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmclassification/v0/hrnet/hrnet-w30_3rdparty_8xb32_in1k_20220120-8aa3832f.pth
Layer                                   Output Shape                                      Param #             
Conv2d                         | Output Shape: torch.Size([1, 64, 320, 320])                      | Parameters: 1728
BatchNorm2d                    | Output Shape: torch.Size([1, 64, 320, 320])                      | Parameters: 128
ReLU                           | Output Shape: torch.Size([1, 64, 320, 320])                      | Parameters: 0
Conv2d                         | Output Shape: torch.Size([1, 64, 160, 160])                      | Parameters: 36864
BatchNorm2d                    | Output Shape: torch.Size([1, 64, 160, 160])                      | Parameters: 128
ReLU                           | Output Shape: torch.Size([1, 64, 160, 160])                      | Parameters: 0
Conv2d                         | Output S

In [18]:
def load_image(img_path, target_size=(640, 640)):
    img = Image.open(img_path).convert('RGB')  # Convert to RGB to ensure 3 channels
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    return img_tensor

# Function to get intermediate layer outputs
def get_intermediate_layers(model, input_image, layer_names):
    activations = {}
    def hook_fn(module, input, output):
        layer_name = module.__class__.__name__
        if layer_name in layer_names:
            activations[layer_name] = output.detach()
    
    hooks = []
    for name, module in model.named_modules():
        if name in layer_names:
            hooks.append(module.register_forward_hook(hook_fn))
    
    # Forward pass
    with torch.no_grad():
        model(input_image)
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    return activations

def display_activation_maps(activations, input_img, n_cols=4):
    n_rows = len(activations)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
    
    for i, (layer_name, activation) in enumerate(activations.items()):
        num_filters = activation.shape[1]
        for col in range(n_cols):
            filter_index = col
            if filter_index < num_filters:
                activation_img = activation[0, filter_index].cpu().numpy()
                activation_img -= activation_img.mean()
                activation_img /= activation_img.std() + 1e-5
                activation_img *= 64
                activation_img += 128
                activation_img = np.clip(activation_img, 0, 255).astype('uint8')
                
                ax = axes[i, col]
                ax.imshow(input_img)
                ax.imshow(activation_img, cmap='viridis', alpha=0.6)
                ax.axis('off')
                
        # Set layer name as the title for the first column of each row
        axes[i, 0].set_title(layer_name)
    
    plt.show()

In [19]:
image_path = 'data/illustrator_6.jpg'  # test image path
input_image = load_image(image_path)

# Specific layer names for which we want to get the activations
layer_names = ['Basicblock']  # Adjust layer names based on your model
activations = get_intermediate_layers(model, input_image, layer_names)

# Load the input image for display
input_img = np.array(Image.open(image_path).resize((640, 640)))

# Visualize the activation maps
display_activation_maps(activations, input_img)

ValueError: Number of rows must be a positive integer, not 0

<Figure size 1500x1500 with 0 Axes>